In [19]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

In [7]:
loader  = DirectoryLoader(
    path = 'directory_loader_multiple_pdfs',
    glob = '*.pdf',
    loader_cls = PyPDFLoader,
    show_progress=True,
)
document_list = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap = 70
)

data_list = splitter.split_documents(document_list)

embedding  = OllamaEmbeddings(model = 'nomic-embed-text:latest')


100%|██████████| 4/4 [00:00<00:00, 82.64it/s]


In [ ]:
#creation of a vector store
'''If the persist_directory exists and contains the data for the collection (in this case, chroma_vector_store), it will load the collection from disk.

If the collection does not exist in the specified directory, it will create a new collection and save it to persist_directory.'''

vector_store = Chroma(
    
    embedding_function = embedding,

    collection_name = 'chroma_vector_store',

    persist_directory = './chroma_store'
    
)

C:\Users\ACER\AppData\Local\Temp\ipykernel_17908\3393577737.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


In [11]:
#add documents to vector store

vector_store.add_documents(data_list)

['a9ac1f89-55e2-4861-b111-07a38d045eff',
 '0960a4d6-8de8-428c-8929-9e2fbb3a495f',
 'f3069d6b-e74b-4dcd-88d8-04b1187ff38c',
 '6177734b-51e8-4b8e-96d7-b0d51bcafcd3',
 '1cc9edbe-0b3b-4aa7-9149-860d5dd0556f',
 '1cf8ae09-ff18-42ea-acba-d44950f7a8a1',
 'cfa8e91a-4db7-42b8-8283-ac4bdc5dd8f5',
 '980f17a9-2716-411b-9fb7-20b4f2ac2d36',
 'c1383945-1993-47ce-ba61-8ad499288bc7',
 '0e651775-b68f-4530-88bf-89b3b5855eee']

In [ ]:
#

vector_store.get(include = ['embeddings','documents','metadatas'])

{'ids': ['a9ac1f89-55e2-4861-b111-07a38d045eff',
  '0960a4d6-8de8-428c-8929-9e2fbb3a495f',
  'f3069d6b-e74b-4dcd-88d8-04b1187ff38c',
  '6177734b-51e8-4b8e-96d7-b0d51bcafcd3',
  '1cc9edbe-0b3b-4aa7-9149-860d5dd0556f',
  '1cf8ae09-ff18-42ea-acba-d44950f7a8a1',
  'cfa8e91a-4db7-42b8-8283-ac4bdc5dd8f5',
  '980f17a9-2716-411b-9fb7-20b4f2ac2d36',
  'c1383945-1993-47ce-ba61-8ad499288bc7',
  '0e651775-b68f-4530-88bf-89b3b5855eee'],
 'embeddings': array([[ 0.07717554,  0.09940337, -0.16076249, ..., -0.02403516,
         -0.04476668, -0.01540614],
        [ 0.06954544,  0.08045968, -0.14963539, ..., -0.056909  ,
         -0.06617349,  0.00544581],
        [ 0.06440519,  0.06564847, -0.12002006, ..., -0.04753906,
         -0.06959679, -0.01387606],
        ...,
        [ 0.04232399,  0.09997242, -0.15869728, ..., -0.05635781,
         -0.04079562, -0.00146791],
        [ 0.05576308,  0.08182313, -0.17179872, ..., -0.03085197,
         -0.03499945, -0.01657544],
        [ 0.08746433,  0.07263037, 

In [18]:
#first way of similarity search

data = vector_store.similarity_search(

    query='what happens at light speed',

    k=2
)

for i in data:
    
    print(f"{i}\n")

page_content='Light Speed and Mass Increase
Real-World Examples and Conclusion
Particle accelerators, like CERN's Large Hadron Collider, experience relativistic mass effects daily.
Subatomic particles moving at near-light speeds become much heavier than when at rest. In conclusion, light
speed sets a universal speed limit, and the mass increase ensures that no object with mass can reach or
exceed it, preserving the structure of spacetime itself.' metadata={'creationdate': 'D:20250426191014', 'creator': 'PyPDF', 'page': 3, 'page_label': '4', 'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'source': 'directory_loader_multiple_pdfs\\light_speed_mass_increase.pdf', 'total_pages': 4}

page_content='Light Speed and Mass Increase
Introduction to Light Speed and Special Relativity
Light speed, approximately 299,792 kilometers per second, is a fundamental constant in physics. Albert
Einstein's theory of Special Relativity introduced the idea that the laws of physics are the same for a

In [ ]:
#second way of similar retrival using retriever

retriever = vector_store.as_retriever(search_type = 'similarity',search_kwargs={"k":2})

result = retriever.invoke('what happens at light speed')

for i in result:
    
    print(f"{i.page_content}\n")


Light Speed and Mass Increase
Real-World Examples and Conclusion
Particle accelerators, like CERN's Large Hadron Collider, experience relativistic mass effects daily.
Subatomic particles moving at near-light speeds become much heavier than when at rest. In conclusion, light
speed sets a universal speed limit, and the mass increase ensures that no object with mass can reach or
exceed it, preserving the structure of spacetime itself.

Light Speed and Mass Increase
Introduction to Light Speed and Special Relativity
Light speed, approximately 299,792 kilometers per second, is a fundamental constant in physics. Albert
Einstein's theory of Special Relativity introduced the idea that the laws of physics are the same for all
non-accelerating observers and that the speed of light is independent of the motion of the light source.



In [20]:
#add more documents in existing store

vector_store.add_documents(
    [Document(
        page_content="Hello world", 
        metadata={"source": "hello.pdf"})])

['9d2e4b6f-69c0-48b7-adb8-9c2cdb6bd63d']

In [23]:
result = vector_store.similarity_search(

    query="Hello world",

    k=1
)

print(result[0].page_content)

Space Science Topics
How Space-Time Bends Around Mass
Space-time is like a fabric that fills the whole universe. Big things like planets and stars sit on this
fabric and bend it. Think about placing a heavy ball on a stretched-out rubber sheet. The ball makes
a dent, and smaller balls roll toward it. In the same way, planets and stars bend space-time around
them, and this bending is what we feel as gravity. When the Sun bends space-time, the Earth


In [22]:
#if i want to delete a document

vector_store.delete(ids = ['9d2e4b6f-69c0-48b7-adb8-9c2cdb6bd63d'])
